In [2]:
line_layer_name = "North_American_Rail_Network_Lines"
line_feature_class = r"C:\Users\natha\Documents\ArcGIS\Projects\ClimateRailnet\Data\NTAD_North_American_Rail_Network_Lines_-2671630235023475291\North_American_Rail_Network_Lines.shp"
node_layer_name = "North_American_Rail_Network_Nodes"
faf_layer_name = "2017_CFS_Metro_Areas_with_FAF"
join_layer_name = "North_American_Rail_Network_Nodes_with_FAF"

In [4]:
import networkx as nx

import os
arcpy.env.workspace = arcpy.env.workspace.replace(os.sep, '/')
os.chdir(os.path.join(arcpy.env.workspace, '../'))

# Create Graph

In [ ]:
arcpy.management.Delete(join_layer_name)
arcpy.analysis.SpatialJoin(node_layer_name, faf_layer_name, join_layer_name, "JOIN_ONE_TO_MANY", "KEEP_COMMON", match_option="INTERSECT")

In [34]:
arcpy.management.JoinField(line_layer_name, "FRFRANODE", join_layer_name, "FRANODEID", ['FAF_Zone'], index_join_fields="NEW_INDEXES")
arcpy.management.AlterField(line_layer_name, 'FAF_Zone', 'FR_FAF_Zone', 'FR_FAF_Zone')
arcpy.management.JoinField(line_layer_name, "TOFRANODE", join_layer_name, "FRANODEID", ['FAF_Zone'], index_join_fields="NEW_INDEXES")
arcpy.management.AlterField(line_layer_name, 'FAF_Zone', 'TO_FAF_Zone', 'TO_FAF_Zone')

<Result 'North_American_Rail_Network_Lines'>

In [59]:
railnet = nx.Graph()

In [60]:
node_list = []
with arcpy.da.SearchCursor(faf_layer_name, ['FAF_Zone', 'FAF_Zone_1', 'INTPTLAT', 'INTPTLON'] ) as cursor:
    for faf_id, name, lat, lon in cursor:
        node_list.append((int(faf_id), {'faf_id': int(faf_id), 'name': name, 'lat': float(lat), 'lon': float(lon)}))
railnet.add_nodes_from(node_list)    

In [63]:
node_fields = [f.name for f in arcpy.ListFields(line_layer_name)]
links = {}
with arcpy.da.SearchCursor("North_American_Rail_Network_Lines", ['FR_FAF_Zone', 'TO_FAF_Zone'], 'FR_FAF_Zone <> TO_FAF_Zone') as cursor:
    for fr, to in cursor:
        pair = (int(fr), int(to))
        if pair not in links:
            links[pair] = 0
        links[pair] += 1
#         links[(int(fr), int(to))] = 


railnet.add_weighted_edges_from([(fr, to, links[(fr, to)]) for fr,to in links])

In [65]:
nx.write_gml(railnet,'Data/faf_railnet.gml')

# Load onto map

In [3]:
railnet = nx.read_gml('Data/faf_railnet.gml')

In [5]:
sr = arcpy.Describe(faf_layer_name).spatialReference
arcpy.env.outputCoordinateSystem = sr

In [6]:
sr

name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [14]:
faf_nodes_fc = f'{arcpy.env.workspace}/faf_nodes'
faf_edges_fc = f'{arcpy.env.workspace}/faf_edges'
arcpy.management.Delete('faf_nodes')
arcpy.management.Delete('faf_edges')

<Result 'true'>

In [15]:
arcpy.conversion.ExportFeatures('Data/faf_network_nodes.shp', faf_nodes_fc)
arcpy.conversion.ExportFeatures('Data/faf_network_edges.shp', faf_edges_fc)

<Result 'C:\\Users\\natha\\Documents\\ArcGIS\\Projects\\ClimateRailnet\\MyProject.gdb\\faf_edges'>

361
131
481
241
221
482
11
251
331
441
362
451
371
171
181
211
391
392
393
483
484
394
81
261
485
183
65
262
372
452
91
486
182
121
201
291
473
222
487
321
61
212
471
122
551
271
12
472
223
92
341
363
423
401
311
123
101
342
421
41
422
411
532
373
19
20
49
50
69
89
99
109
129
139
159
160
179
189
190
209
219
229
230
249
259
269
279
280
299
300
319
329
339
350
369
379
380
399
409
419
429
459
460
479
489
499
500
519
539
540
559
560
511
364
62
491
488
63
64
132
531
172
292
124
42
402
151
512
111
242
513
202
